In [127]:
import numpy as np
import pandas as pd

# Definición del modelo en python


Creamos la clase nodo sobre los que implementaremos las neuronas

In [128]:
class NetNode(object):
    """ Base class that represents a node in a neural net """

    def __init__(self):
        self.inputs = []
        self.weights = []
        self.value = None
    def __str__(self) -> str:
        i = '-'.join(str(x) for x in self.inputs) + ""
        w = '-'.join(str(x) for x in self.inputs) + ""
        return str(self.value) + " I:[" + "".join(map(str,self.inputs)) + "] W:[" + "".join(map(str,self.weights)) +"]"

Clases para la configuración de las neuronas

In [129]:
class Sigmoide():
    def act(z):
        return 1/(1+np.exp(-z))
    def prime(z):
        return Sigmoide.act(z) * (1- Sigmoide.act(z))  

In [130]:
class ReLu():
    def act(z):
        return max(0, z)
    
    def prime(z):
        return 1 if z > 0 else 0

A continuación crearemos la estructura de la red

In [131]:
class Network(object):
    """ Main class to construct ant train neural networks """

    def __init__(self, layers, funcionDeActivacion):
        """ 
        Parameters
        ----------
        layers:
            A list that represents the neurons and layers of the network.
            For example, [2, 3, 1] represents a network with 3 layers:
            - input layer: 2 neurons.
            - hidden layer: 3 neurons.
            - output layer: 1 neuron. 
        """

        # Funciones
        self.funcionDeActivacion=funcionDeActivacion
        #Creamos la estructura
        self.net = [[NetNode() for _ in range(size)] for size in layers]

        #Realizamos las conexiones y establecemos el peso por omisión a 0
        sizes = len(layers)
        # Make connections
        for layer in range(1, sizes):
            for node in self.net[layer]:
                for unit in self.net[layer - 1]:
                    node.inputs.append(unit)
                    node.weights.append(0)

    #Predicción para los datos de entrada
    def predict(self, input_data):
        inputs = self.net[0]

        # Initialize inputs
        for v, n in zip(input_data, inputs):
            n.value = v

        # Forward step, comenzamos en la priemra capa oculta
        for layer in self.net[1:]:
            for node in layer:
                in_val = [n.value for n in node.inputs]
                #Aplicamos los pesos de los valores de entrada
                unit_value = np.dot(in_val, node.weights)
                node.value = self.funcionDeActivacion[self.net.index(layer)-1].act(unit_value)

        #Como resultado de la predicción devolvemos los índices del array de salida
        

        #Damos como salida el nodo ccuyo valor sea máximo. O lo que es lo mismo,
        # devolvemos el nodo que caracteriza a la clase de los datos de entrada.
        outputs = self.net[-1]     
        result = outputs.index(max(outputs, key=lambda node: node.value))      
        return result

    def accuracy(self, examples):
        correct = 0

        for x_test, y_test in examples:
            prediction = self.predict(x_test)

            if (y_test[prediction] == 1):
                correct += 1

        return correct / len(examples)

    #Eta es la tasa dxe aprendizaje
    #epochs el número de iteraciones
    def backpropagation(self, eta, examples, epochs):
        inputs = self.net[0]
        outputs = self.net[-1]
        layer_size = len(self.net)

        # Initialize weights con un valor aleatorio para todas las capas menos
        # para la de entrada
        for layer in self.net[1:]:
            for node in layer:
                node.weights = [np.random.uniform()
                                for _ in range(len(node.weights))]

        for epoch in range(epochs):
            for x_train, y_train in examples:
                # Initialize inputs
                for value, node in zip(x_train, inputs):
                    node.value = value

                # Forward step
                for layer in self.net[1:]:
                    # print("En forward utilizo " + str(self.funcionDeActivacion[self.net.index(layer)-1]) + " para la capa " + str(self.net.index(layer))) 
                    for node in layer:
                        in_val = [n.value for n in node.inputs]
                        unit_value = np.dot(in_val, node.weights)
                        node.value = self.funcionDeActivacion[self.net.index(layer)-1].act(unit_value)

                # Initialize delta
                delta = [[] for _ in range(layer_size)]

                # Error for the MSE cost function
                err = [y_train[i] -
                       outputs[i].value for i in range(len(outputs))]

                delta[-1] = [self.funcionDeActivacion[self.net.index(layer)-1].prime(outputs[i].value) * err[i]
                             for i in range(len(outputs))]

                # Backward step
                hidden_layers = layer_size - 2
                for i in range(hidden_layers, 0, -1):

                    layer = self.net[i]
                    n_layers = len(layer)
                    # print("En Backward utilizo " + str(self.funcionDeActivacion[i-1]) + " para la capa " + str(self.net.index(layer))) 

                    # Weights from the last layer
                    w = [[node.weights[l] for node in self.net[i + 1]]
                         for l in range(n_layers)]

                    delta[i] = [self.funcionDeActivacion[i-1].prime(
                        layer[j].value) * np.dot(w[j], delta[i + 1]) for j in range(n_layers)]

                # Update weights
                for i in range(1, layer_size):
                    layer = self.net[i]
                    in_val = [node.value for node in self.net[i - 1]]
                    n_layers = len(self.net[i])
                    for j in range(n_layers):
                        layer[j].weights = np.add(
                            layer[j].weights, np.multiply(eta * delta[i][j], in_val))

            print(f"epoch {epoch}/{epochs} | total error={np.sum(err)/len(examples)}")



# Prueba del modelo implementado

In [132]:
from sklearn import datasets
from sklearn.preprocessing import normalize
from sklearn.model_selection import train_test_split
from keras.utils import np_utils

In [133]:
# import data to play with
iris_X, iris_y = datasets.load_iris(return_X_y=True)

In [134]:
# First 10 elements of input data
iris_X[:10]

array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2],
       [4.6, 3.1, 1.5, 0.2],
       [5. , 3.6, 1.4, 0.2],
       [5.4, 3.9, 1.7, 0.4],
       [4.6, 3.4, 1.4, 0.3],
       [5. , 3.4, 1.5, 0.2],
       [4.4, 2.9, 1.4, 0.2],
       [4.9, 3.1, 1.5, 0.1]])

In [135]:
# First 10 elements of output data
iris_y[:10]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [136]:
iris_x_normalized = normalize(iris_X, axis=0)

In [137]:
# Creating train and test data
'''
80% -- train data
20% -- test data
'''
X_train, X_test, y_train, y_test = train_test_split(
    iris_x_normalized, iris_y, test_size=0.2, shuffle=True)

In [138]:
# Convert classes from categorical ('Setosa', 'Versicolor', 'Virginica')
# to numerical (0, 1, 2) and then to one-hot encoded ([1, 0, 0], [0, 1, 0], [0, 0, 1]).
'''
[0]--->[1 0 0]
[1]--->[0 1 0]
[2]--->[0 0 1]
'''
y_train = np_utils.to_categorical(y_train, num_classes=3)
y_test = np_utils.to_categorical(y_test, num_classes=3)

In [139]:
examples = []
for i in range(len(X_train)):
    examples.append([X_train[i], y_train[i]])

In [140]:
net = Network([4, 7, 3], [Sigmoide,ReLu])
net.backpropagation(0.1, examples, 500)

epoch 0/500 | total error=-4.651952546939203e-05
epoch 1/500 | total error=-4.644555092574975e-05
epoch 2/500 | total error=-4.6376317941116087e-05
epoch 3/500 | total error=-4.631278828678836e-05
epoch 4/500 | total error=-4.625584776388955e-05
epoch 5/500 | total error=-4.620627173456219e-05
epoch 6/500 | total error=-4.616468565725821e-05
epoch 7/500 | total error=-4.613151919120248e-05
epoch 8/500 | total error=-4.6106952419867435e-05
epoch 9/500 | total error=-4.609085272157971e-05
epoch 10/500 | total error=-4.608270086112337e-05
epoch 11/500 | total error=-4.608150503676162e-05
epoch 12/500 | total error=-4.608570195785029e-05
epoch 13/500 | total error=-4.609304463555943e-05
epoch 14/500 | total error=-4.610047755373205e-05
epoch 15/500 | total error=-4.6104001381160056e-05
epoch 16/500 | total error=-4.609853153842621e-05
epoch 17/500 | total error=-4.607775788391943e-05
epoch 18/500 | total error=-4.603401663464679e-05
epoch 19/500 | total error=-4.595819038008264e-05
epoch 2

In [141]:
examples = []
for i in range(len(X_test)):
    examples.append([X_test[i], y_test[i]])


In [142]:
accuracy = net.accuracy(examples)
print(f"Accuracy: {accuracy}")

Accuracy: 0.9


In [143]:
prediction = net.predict(X_test[1])
print(f"Desired output: {y_test[1]}")
print(f"Index of output: {prediction}")

Desired output: [0. 0. 1.]
Index of output: 2


In [144]:
#print(net.net)
df = pd.DataFrame(net.net)
print(df)


                                                   0  \
0                      0.07886412818583192 I:[] W:[]   
1  0.6079603553260472 I:[0.07886412818583192 I:[]...   
2  0 I:[0.6079603553260472 I:[0.07886412818583192...   

                                                   1  \
0                       0.0661014913510784 I:[] W:[]   
1  0.4082531171910942 I:[0.07886412818583192 I:[]...   
2  0.735647828684781 I:[0.6079603553260472 I:[0.0...   

                                                   2  \
0                      0.09838574584787062 I:[] W:[]   
1  0.6354483587169235 I:[0.07886412818583192 I:[]...   
2  0.8414844605876992 I:[0.6079603553260472 I:[0....   

                                                   3  \
0                       0.1150242403183535 I:[] W:[]   
1  0.5110481129110992 I:[0.07886412818583192 I:[]...   
2                                               None   

                                                   4  \
0                                           